# Import Libraries

In [1]:
import numpy as np 

import matplotlib.pyplot as plt


from numpy import *

from sklearn import datasets 
from sklearn.metrics import mean_squared_error  

from sklearn.model_selection import train_test_split 
from sklearn import metrics 

#keras 

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers

from keras.layers import Dense
from keras.models import Sequential

from tensorflow.keras.callbacks import Callback

import random


from sklearn.metrics import roc_curve, auc

#https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0253217

fpath = 'C:\\Dropbox\\Variance\\UNSW\\ZZSC5836\\'

# Functions

In [2]:
def read_data(run_num):
    #Source - raw and processed data :  https://github.com/sydney-machine-learning/Bayesianneuralnet_stockmarket/tree/master/code/dataset
    # five inputs (window size of 5) for 5 steps ahead (MMM dataset) https://finance.yahoo.com/quote/MMM/
    #code to process raw data: https://github.com/sydney-machine-learning/Bayesianneuralnet_stockmarket/blob/master/code/data.py
    data_in = genfromtxt(fpath+"raw_data\\MMM8_train.txt", delimiter=" ")
    data_inputx = data_in[:,0:5] # all features 0, 1, 2, 3, 4, 5, 6, 7 

    #https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer
    #transformer = Normalizer().fit(data_inputx)  # fit does nothing.
    #data_inputx = transformer.transform(data_inputx)
    data_inputy = data_in[:,5:10] # this is target - so that last col is selected from data

    x_train, x_test, y_train, y_test = train_test_split(data_inputx, data_inputy, test_size=0.40, random_state=run_num)

    return x_train, x_test, y_train, y_test

 
    
def nn(x_train, x_test, y_train, y_test, type_model, hidden):
 

    timesteps = 5 # window size
    #steps_ahead = 5 
    steps_ahead = 5 # prediction horizon 1 

    if type_model ==0: #keras Adam
        nn = keras.Sequential()
        nn.add(layers.Dense(hidden, input_dim=timesteps, activation='relu'))
        nn.add(layers.Dense(steps_ahead, activation='sigmoid'))
        nn.compile(loss=keras.losses.binary_crossentropy,optimizer='adam', metrics=[keras.metrics.RootMeanSquaredError(), keras.metrics.mae,keras.metrics.mape])
         
    else:
        print('no model')    
  
    history = nn.fit(x_train, y_train, epochs=500, batch_size=32, verbose=0)#callbacks=[fa_test_his])
    y_pred_test = nn.predict(x_test)
    n = len(y_test)
    MAE = sum(np.abs(y_test - y_pred_test)) / n 
    RMSE_horizon1 = np.sqrt(sum(np.square(y_test[:,0] - y_pred_test[:,0])) / n) # prediction horizon 1

    RMSE = np.sqrt(sum(np.square(y_test - y_pred_test)) / n) 
    MAPE=sum(np.abs((y_test - y_pred_test) / (y_test + 1e-6))) / n * 100  
   
    return RMSE, RMSE_horizon1, y_test, y_pred_test, MAPE




# Main

In [3]:
max_expruns = 2

Adam_all_acc = np.zeros(max_expruns) 
mape_all = np.zeros(max_expruns) 
rmse_all = np.zeros(max_expruns) 
rmse_horizon1_all = np.zeros(max_expruns) 
    
max_hidden = 10

for hidden in range(6,max_hidden, 2):

    for run_num in range(0,max_expruns): 

        x_train, x_test, y_train, y_test = read_data(0)   

        rmse, RMSE_horizon1, y_test, y_pred_test, mape = nn(x_train, x_test, y_train, y_test, 0, hidden) 
            
        mape_all[run_num] = mape
        rmse_all[run_num] = rmse
        rmse_horizon1_all[run_num] = RMSE_horizon1

    print(f"{rmse_all=}, {mape_all=}, {hidden=}, Accuracy, MAPE all runs")
    #print(f"{np.std(Adam_all_acc)=}, {hidden=} std")
    print(f"{np.mean(mape_all)=}, {hidden=} mape")
    print(f"{np.mean(rmse_all)=}, {hidden=} rmse")
    print(f"{np.mean(rmse_horizon1_all)=}, {hidden=} rmse_horizon1")

    # Put rmse values into context
    range_target = np.max(y_train) - np.min(y_train)
    std_dev_target = np.std(y_train)
    proportion_of_range_full = np.mean(rmse_all) / range_target
    proportion_of_std_dev_full = np.mean(rmse_all) / std_dev_target
    proportion_of_range_first = np.mean(rmse_horizon1_all) / range_target
    proportion_of_std_dev_first = np.mean(rmse_horizon1_all) / std_dev_target
    print(f"{proportion_of_range_full=}, {proportion_of_std_dev_full=}")
    print(f"{proportion_of_range_first=}, {proportion_of_std_dev_first=}")


#next try a paragraph to describe your results and discuss which models are better to use.
#repeat for another dataset
# you can save results to a file as well


11/11 [==============================] - 0s 598us/step
rmse_all=array([0.08129776, 0.08015849]), mape_all=array([292.14145439, 294.99881429]), hidden=6, Accuracy, MAPE all runs
np.mean(mape_all)=293.57013434115254, hidden=6 mape
np.mean(rmse_all)=0.08072812545481084, hidden=6 rmse
np.mean(rmse_horizon1_all)=0.03461283946723716, hidden=6 rmse_horizon1
proportion_of_range_full=0.08081394986957233, proportion_of_std_dev_full=0.3174495299622255
proportion_of_range_first=0.034649637382136986, proportion_of_std_dev_first=0.13610906431467998
11/11 [==============================] - 0s 603us/step
rmse_all=array([0.08013774, 0.04540305]), mape_all=array([291.33346577,  80.06800257]), hidden=8, Accuracy, MAPE all runs
np.mean(mape_all)=185.7007341667431, hidden=8 mape
np.mean(rmse_all)=0.06277039496745437, hidden=8 rmse
np.mean(rmse_horizon1_all)=0.02556539299054058, hidden=8 rmse_horizon1
proportion_of_range_full=0.0628371279973876, proportion_of_std_dev_full=0.24683382979226798
proportion_of_r

In [4]:
print('x_train shape:', x_train.shape, 'y_train shape:', y_train.shape)
print('x_train')
print(x_train[:2])
print('y_train')
print(y_train[:2])

x_train shape: (482, 5) y_train shape: (482, 5)
x_train
[[0.149406 0.1466   0.144777 0.144105 0.150437]
 [0.136047 0.136431 0.148135 0.146696 0.143866]]
y_train
[[0.148231 0.149622 0.144105 0.145736 0.142906]
 [0.138158 0.148566 0.147751 0.151972 0.157056]]


# END

# Explanation

Summary of How the Code Works:

    Data Reading and Preprocessing:
        The function read_data reads input data from a file, splits it into features (data_inputx) and targets (data_inputy), and then divides these into training and testing sets using train_test_split.

    Neural Network Model Definition and Training:
        The function nn defines and trains a neural network model. The model is a sequential Keras model with one hidden layer (Dense) and an output layer. It uses 'relu' activation for the hidden layer and 'sigmoid' for the output layer.
        The model is compiled with a binary cross-entropy loss function and the Adam optimizer.
        It is trained on the training data and then used to predict on the test data. Various performance metrics like Mean Absolute Error (MAE), Root Mean Square Error (RMSE), and Mean Absolute Percentage Error (MAPE) are calculated.

    Main Execution Loop:
        The main function runs several experiments (max_expruns) with varying numbers of hidden neurons (max_hidden) in the neural network. For each configuration, it evaluates the model's performance using the RMSE metric.
        The results (RMSE values) across different runs and configurations are printed out for analysis.

Variable Descriptions:

    run_num: A parameter in read_data function, likely used to set a random seed for reproducibility.
    data_in: The dataset loaded from a file.
    data_inputx: The input features extracted from data_in.
    data_inputy: The target values (labels) extracted from data_in.
    x_train, x_test, y_train, y_test: Data split into training and testing sets.
    type_model: A parameter in the nn function, used to select the type of model (only '0' is used, representing Keras with Adam optimizer).
    hidden: The number of neurons in the hidden layer of the neural network.
    nn: The neural network model.
    timesteps, steps_ahead: Constants defining the window size and prediction horizon respectively.
    Adam_all: An array to store RMSE values for different runs.
    max_expruns, max_hidden: Variables controlling the number of experiments and the range of hidden neurons to test.

Shapes of Input and Target Factors:

    Input (Features) Shape: The input data (data_inputx) is derived from the dataset with a shape of [number of samples, 5]. This shape is because the model uses a window size of 5 timesteps.
    Target (Labels) Shape: The target data (data_inputy) is also derived from the dataset with a shape of [number of samples, 5]. This shape is because the model predicts values 5 steps ahead, as indicated by steps_ahead.

The target factor has this shape to match the model's output, which is predicting values for 5 future steps, aligning with the problem of predicting future stock prices for a given number of future time steps.